# Healthcare

## データ解析
### 手順メモ
1. データを読み込む
2. データをサンプルごとに分割する
3. デシリアライズするなどして，pythonに取り込む
4. csvファイルに書き込む
5. 各ポイントのデータをグラフ化する
6. 全身のボーンをつないだグラフを描画する
7. 時間を指定することでリアルタイムで変化を確認できるようにする

#### 初期設定

In [56]:
import json
from logging import getLogger, StreamHandler, DEBUG

logger = getLogger(__name__)
handler = StreamHandler()
# handler.setLevel(level=DEBUG)
logger.setLevel(level=DEBUG)
logger.addHandler(handler)
logger.propagate = False


def get_string_list_from_file_path(file_path):
    with open(file_path, "r", encoding='utf-8_sig') as file:
        string_list = file.readlines()
        print(string_list[0])

    return string_list


def get_obj_from_json_str(json_string):
    obj = json.loads(json_string)
    return obj

In [57]:
path = r"C:\Users\dai\Downloads\healthcare\log_2021_02_01_15_43_43.json"
json_string_list = get_string_list_from_file_path(path)
objs = []

for s in json_string_list:
    objs.append(get_obj_from_json_str(s))

print(objs[0])

{"time":"02/01/2021 15:43:43.983","angles":[{"part":"Neck","x":-39.19276809692383,"y":150.63330078125,"z":24.646284103393556},{"part":"Hip","x":-96.56009674072266,"y":145.3456573486328,"z":-4.545019149780273},{"part":"LeftKnee","x":-10.801756858825684,"y":-18.317485809326173,"z":3.6142215728759767},{"part":"RightKnee","x":-9.105908393859864,"y":-28.30419921875,"z":4.933361053466797}],"boneTransformData":{"positions":[{"x":3.4993014335632326,"y":-0.22930222749710084,"z":2.343693971633911},{"x":3.4993014335632326,"y":-0.22930222749710084,"z":2.343693971633911},{"x":3.582071542739868,"y":-0.24760881066322328,"z":2.4032514095306398},{"x":3.6960060596466066,"y":-0.6499114632606506,"z":2.3532748222351076},{"x":3.66768741607666,"y":-1.0982532501220704,"z":2.438814878463745},{"x":3.7725746631622316,"y":-1.1579517126083375,"z":2.3513777256011965},{"x":3.8175575733184816,"y":-1.159936785697937,"z":2.310729742050171},{"x":3.4170162677764894,"y":-0.2607295513153076,"z":2.289156198501587},{"x":3.46